# 第2回最適化勉強会  2020/6/4
### 今日の目標：Pythonでマス目の問題を解く
- N Queen
- 魔方陣
- 日直

## N Queen
- 4×4のマスにQueenを4つ置く
- Queenは互いに取れないようにする
  - 1行に1つ
  - 1列に1つ
  - 斜めに1つ以下

In [ ]:
import pandas as pd
n = 4
df = pd.DataFrame([(i, j, i + j, i - j + n - 1) for i in range(n)
                   for j in range(n)], columns=list('YXRL'))
df[:2]

In [ ]:
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
fs = FloatSlider(0, min=0, max=2 * n - 2, step=1, continuous_update=False)
@interact(col=df.columns, val=fs)
def show_df(col, val):
    df2 = pd.concat([df, pd.Series([0] * (n * n), name='On')], 1)
    df2.loc[df2[col] == val, 'On'] = 1
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.imshow(df2.On.values.reshape(n, n), 'gray')

In [ ]:
# dfの行は、4×4の各マスに対応
# 先頭行は左上、2行目はその右隣り
# X Y R L を確認してみる
df[:2]

In [ ]:
# 変数列の追加
# 変数が1なら、そのマスにQueenを置くことを意味する
from ortoolpy import addbinvars
addbinvars(df)
df[:2]

In [ ]:
# モデル作成＆ソルバー実行
from ortoolpy import model_min, lpSum, addvals
m = model_min()
for col in 'YX':
    for i in range(n):
        m += lpSum(df[df[col] == i].Var) == 1
for col in 'RL':
    for i in range(2 * n - 1):
        m += lpSum(df[df[col] == i].Var) <= 1
%time m.solve()

In [ ]:
# 結果の確認（白にQueenを置く）
addvals(df)
plt.imshow(df.Val.values.reshape(n, n), 'gray');

### ポイント
- DataFrameに変数の列 Var を追加する
  - 0-1変数の場合は、`addbinvars(df)`とする
- 目的関数なしの場合、便宜的に最小化モデルとする
- DataFrameで必要な変数を絞り込む
- 変数を含む式の合計は、`sum`ではなく`lpSum`を使う
- solveで解を求める
- モデルのstatusが1なら最適解が得られている
- DataFrameに結果の列 Val を追加する

In [ ]:
# 解空間の広さ（制約条件を無視した変数の取りうる組み合わせ）
2**(n * n)

### やってみよう
`n = 10` にしてやり直してみよう

---
## 魔方陣
- 3×3のマスに「1から9までの数字」を埋める
- 各行、各列、斜めの和は、いずれも15

In [ ]:
n = 3
df = pd.DataFrame([(i, j, k + 1) for i in range(n) for j in range(n)
                   for k in range(n * n)], columns=list('YXN'))
addbinvars(df)
df[:2]

In [ ]:
# 1つのマスに1つの数字
m = model_min()
for _, gr in df.groupby(['X', 'Y']):
    m += lpSum(gr.Var) == 1

In [ ]:
gr

In [ ]:
# 各数字は1つだけ
for _, gr in df.groupby('N'):
    m += lpSum(gr.Var) == 1

In [ ]:
gr

In [ ]:
# 和を15に
# 内積（lpDot）を使おう
from ortoolpy import lpDot
for col in 'YX':  # 行と列
    for i in range(n):
        m += lpDot(df[df[col] == i].Var, df.N) == 15
m += lpDot(df[df.X - df.Y == 0].Var, df.N) == 15  # 左斜め（＼）
m += lpDot(df[df.X + df.Y == 2].Var, df.N) == 15  # 右斜め（／）
%time m.solve()

In [ ]:
addvals(df)
df[df.Val > 0]

In [ ]:
df[df.Val > 0].N.values.reshape(n, n)

---
## 日直
- 月火水木金を、hirokiky, kameko, kenken, konie, nanaで分担する
- kamekoさんは水曜ができない
- kenkenさんは金曜にやりたい

In [ ]:
week = '月火水木金'
names = 'hirokiky kameko kenken konie nana'.split()
df = pd.DataFrame([(w, n) for w in week for n in names],
                  columns=['Week', 'Name'])
addbinvars(df)
df[:2]

In [ ]:
# 各曜日に1人
from ortoolpy import model_max
m = model_max()  # 最大化問題
for _, gr in df.groupby(['Week']):
    m += lpSum(gr.Var) == 1

In [ ]:
# 各人が1回
for _, gr in df.groupby(['Name']):
    m += lpSum(gr.Var) == 1

In [ ]:
# kamekoさんは水曜ができない
m += lpSum(df.query('Week == "水" & Name == "kameko"').Var) == 0

In [ ]:
# kenkenさんは金曜にやりたい
m += lpSum(df.query('Week == "金" & Name == "kenken"').Var)

In [ ]:
m.solve()

In [ ]:
addvals(df)
df[df.Val > 0]

終わり